<a href="https://colab.research.google.com/github/Quadragonsaurus/ArielACE/blob/main/NekoFi_SuperLazy_Cafe_TI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Installing the required
#@markdown Please don't check this first, I haven't optimized it yet.
use_drive = False #@param {type:"boolean"}

if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive
else:
  %cd /content

!pip install -q -U gdown
url = "https://huggingface.co/andite/models/resolve/main/nai-wd.ckpt" #@param {type:"string"}

if use_drive:
  !gdown {url} -O /content/drive/MyDrive/sd_text_inversion/
else:
  !gdown {url} -O /content/sd_text_inversion/
 
if use_drive:
  %cd /content/drive/MyDrive
  !mkdir sd_text_inversion
  %cd sd_text_inversion
  !mkdir modelsbackup
  !mkdir stable-textual-inversion-cafe
  !mkdir logs
  !mkdir output
  !rm -rf stable-textual-inversion-cafe
  %cd /content/drive/MyDrive/sd_text_inversion
  !git clone https://github.com/Raearn/stable-textual-inversion-cafe.git
  %cd stable-textual-inversion-cafe
else:
  %cd /content
  !mkdir sd_text_inversion
  %cd sd_text_inversion
  !mkdir modelsbackup
  !mkdir stable-textual-inversion-cafe
  !mkdir logs
  !mkdir output
  !rm -rf stable-textual-inversion-cafe
  %cd /content/sd_text_inversion
  !git clone https://github.com/Raearn/stable-textual-inversion-cafe.git
  %cd stable-textual-inversion-cafe

import os.path
from pathlib import Path
print("Done.")

if use_drive:
  %cd /content/drive/MyDrive/sd_text_inversion/stable-textual-inversion-cafe
else:
  %cd /content/sd_text_inversion/stable-textual-inversion-cafe

!pip install omegaconf einops pytorch-lightning==1.6.5 test-tube transformers kornia -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install torchtext==0.13.1
!pip install -e .

print("Done. run the cell under this to start training")


/content
Downloading...
From: https://huggingface.co/andite/models/resolve/main/nai-wd.ckpt
To: /content/sd_text_inversion/nai-wd.ckpt
100% 4.27G/4.27G [01:19<00:00, 53.4MB/s]
/content
mkdir: cannot create directory ‘sd_text_inversion’: File exists
/content/sd_text_inversion
/content/sd_text_inversion
Cloning into 'stable-textual-inversion-cafe'...
remote: Enumerating objects: 706, done.
remote: Counting objects: 100% (703/703), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 706 (delta 409), reused 650 (delta 384), pack-reused 3
Receiving objects: 100% (706/706), 4.35 MiB | 24.61 MiB/s, done.
Resolving deltas: 100% (409/409), done.
/content/sd_text_inversion/stable-textual-inversion-cafe
Done.
/content/sd_text_inversion/stable-textual-inversion-cafe
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transform

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 6.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 47.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pi

In [4]:
#@title Start TI Training
#@markdown Dont't use any space in project_name, you can use underscore (_) instead.<p>
auto = "auto"
project_name = "izu" #@param {type:"string"}
#@markdown If you change the `initializer_words`, make sure you keep the syntax correct.                       
initializer_words = ["girl","face","hair","eyes","headset","clothes"] #@param {type:"raw"}
#@markdown If you are not sure how much vectors you need, you can set `num_vectors_per_token` to `auto` instead of a number. <p>
num_vectors_per_token = auto #@param {type:"raw"}                                               
ti_type = "character" #@param ["character", "artstyle"]
model = "/content/sd_text_inversion/nai-wd.ckpt" #@param {type:"string"}

#@markdown # Dataset part
#@markdown First please choose if you want to use a huggingface/drive link or a path <br>
#@markdown Then specify the `url` or the `dataset_path` <br>

path_type = "path" #@param ["path", "url"]

url = "" #@param {type:"string"}
dataset_path = "/content/train_data" #@param {type:"string"}
#@markdown ## *After setting each values, don't forget to click the run button.*
#@title Downloading dataset and creating training configuration yaml
#@markdown After running this cell, you can still check your configuration if you want to modify anything.
#@markdown It is located in the `/content/sd_text_inversion/stable-textual-inversion-cafe/configs/stable-diffusion` folder, named as `<project_name>.yaml`

import os

# downloading dataset
if (path_type == "url"):
    if (url.startswith("https://drive.google.com")):
        !gdown {url} -O /content/{project_name}
    else:
        !gdown {url} -O /content/{project_name}.zip
    !unzip /content/{project_name}.zip -d /content/sd_text_inversion/Imagesfortraining/{project_name}
    num_images = len(os.listdir('/content/sd_text_inversion/Imagesfortraining/' + project_name))
else:
    num_images = len(os.listdir(dataset_path))

# loading the base config
import yaml
with open("/content/sd_text_inversion/stable-textual-inversion-cafe/configs/stable-diffusion/" + ti_type + ".yaml", "r") as f:
    base_config = yaml.safe_load(f)

# calulate the number of vectors per token from the number of images
if num_vectors_per_token == "auto":
    num_vectors_per_token = round(num_images / 8.5)
    if num_vectors_per_token < 8:
        num_vectors_per_token = 8
    if num_vectors_per_token > 18:
        num_vectors_per_token = 18

# calculating max steps
if num_images <= 110:
    max_steps = 11000
else:
    max_steps = 16000
import math
repeats = math.ceil(max_steps / num_images)

# setting the parameters
base_config["model"]["params"]["personalization_config"]["params"]["initializer_words"] = initializer_words
base_config["model"]["params"]["personalization_config"]["params"]["num_vectors_per_token"] = num_vectors_per_token
base_config["data"]["params"]["train"]["params"]["repeats"] = repeats
base_config["lightning"]["trainer"]["max_steps"] = max_steps
base_config["lightning"]["modelcheckpoint"]["params"]["every_n_train_steps"] = 500
base_config["lightning"]["callbacks"]["image_logger"]["params"]["batch_frequency"] = 2000
base_config["model"]["params"]["log_every_t"] = 500

# saving the config
with open("/content/sd_text_inversion/stable-textual-inversion-cafe/configs/stable-diffusion/" + project_name + ".yaml", "w") as f:
    yaml.dump(base_config, f)
#@title Training
#@markdown This block will train the TI, and when it's done, it will zip the checkpoints automatically. It will be available on your Google Drive, in `/content/sd_text_inversion/output` folder as `<project_name>.zip`
if (ti_type == "character"):
    initializer_word = "character"
elif (ti_type == "artstyle"):
    initializer_word = "illustration"

if (path_type == "url"):
  dataset = "/content/sd_text_inversion/Imagesfortraining/" + project_name
else:
  dataset = dataset_path
config = "/content/sd_text_inversion/stable-textual-inversion-cafe/configs/stable-diffusion/" + project_name + ".yaml"
logs_folder = "/content/sd_text_inversion/logs"
!mkdir /content/sd_text_inversion/logs
!mkdir /content/sd_text_inversion/output

%cd /content/sd_text_inversion/stable-textual-inversion-cafe
!python "main.py" --base {config} -t --no-test --actual_resume {model}  -n {project_name} --gpus 1 --data_root {dataset} --init_word {initializer_word} --logdir {logs_folder}

import os
import time
import datetime

dataset_name = dataset_path.split("/")[-1]
folders = os.listdir(logs_folder)
folders = [folder for folder in folders if folder.startswith(dataset_name)]
creation_times = [os.path.getctime(os.path.join(logs_folder, folder)) for folder in folders]
ordered_folders = [folder for _, folder in sorted(zip(creation_times, folders))]
newest_folder = ordered_folders[-1]

%cd {logs_folder}/{newest_folder}
!zip /content/sd_text_inversion/output/{project_name}.zip checkpoints/*
print("Done! The checkpoints is saved in the output folder.")

mkdir: cannot create directory ‘/content/sd_text_inversion/logs’: File exists
mkdir: cannot create directory ‘/content/sd_text_inversion/output’: File exists
/content/sd_text_inversion/stable-textual-inversion-cafe
Global seed set to 23
Running on GPUs 1
Loading model from /content/sd_text_inversion/nai-wd.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.15.layer_norm2.bias', 'visual_projection.weight', 'vision_model.encoder.layers.18.layer_norm2.bias', 'vision_model.encoder.layers.20.self_attn.out_proj.weight', 'vision_model.encoder.layers.18.self_attn.q_proj.weight', 'vision_model.encoder.layers.10.layer_norm1.weight', 'vision_mod